In [10]:
import tensorflow as tf
import numpy as np
import math
import datetime
import os
import random
# tf.flags.DEFINE_integer('train_steps', 1000,
#                         'Number of training steps')

# set variables
num_epochs = 15
tweet_size = 20
hidden_size = 500
vec_size = 300
batch_size = 512
number_of_layers= 2
number_of_classes= 1
starter_learning_rate = 0.001

tf.reset_default_graph()

# Create a session 
session = tf.Session()

# Inputs placeholders
tweets = tf.placeholder(tf.float32, [None, tweet_size, vec_size], "tweets")
labels = tf.placeholder(tf.float32, [None], "labels")

# Placeholder for dropout
keep_prob = tf.placeholder_with_default(1.0,[], name="keep_prob") 

# make the lstm cells, and wrap them in MultiRNNCell for multiple layers
def lstm_cell():
    cell = tf.contrib.rnn.BasicLSTMCell(hidden_size)
    return tf.contrib.rnn.DropoutWrapper(cell=cell, input_keep_prob=keep_prob, output_keep_prob=keep_prob)

multi_lstm_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(number_of_layers)], state_is_tuple=True)

batch_size_T  = tf.shape(tweets)[0]
zerostate = multi_lstm_cells.zero_state(batch_size_T, dtype=tf.float32)

# Creates a recurrent neural network
_, final_state = tf.nn.dynamic_rnn(multi_lstm_cells, tweets, dtype=tf.float32, initial_state=zerostate)

sentiments = tf.contrib.layers.fully_connected(final_state[-1][-1], num_outputs=number_of_classes, activation_fn=None, weights_initializer=tf.random_normal_initializer(),
  biases_initializer=tf.random_normal_initializer(), scope="fully_connected")

sentiments = tf.squeeze(sentiments, [1])

predictions = tf.nn.sigmoid(sentiments, name="pred_op")

with tf.name_scope("loss"):
    # define cross entropy loss function
    losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=sentiments, labels=labels)
    loss = tf.reduce_mean(losses, name="loss_op")
    #tensorboard summaries
    tf.summary.scalar("loss", loss)


with tf.name_scope("accuracy"):
    # round our actual probabilities to compute error
    accuracy = tf.to_float(tf.equal(tf.to_float(tf.greater_equal(predictions, 0.5)), labels))
    accuracy = tf.reduce_mean(tf.cast(accuracy, dtype=tf.float32), name="acc_op")
    #tensorboard summaries
    tf.summary.scalar("accuracy", accuracy)
    
    
global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           1000, 0.96, staircase=True)    
# define our optimizer to minimize the loss
with tf.name_scope("train"):
    optimizer = tf.train. AdamOptimizer(learning_rate).minimize(loss)

#tensorboard summaries
merged_summary = tf.summary.merge_all()
logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
train_writer = tf.summary.FileWriter(logdir + "-training", session.graph)
test_writer  = tf.summary.FileWriter(logdir + "-testing", session.graph)

# initialize any variables
tf.global_variables_initializer().run(session=session)

# Create a saver for writing training checkpoints.
saver = tf.train.Saver()

# load our data and separate it into tweets and labels
train_tweets = np.load('data_es/train_vec_tweets.npy')
train_labels = np.load('data_es/train_vec_labels.npy')

test_tweets = np.load('data_es/test_vec_tweets.npy')
test_labels = np.load('data_es/test_vec_labels.npy')



steps = int(len(train_tweets)/batch_size)
summary_step = 0

for epoch in range(num_epochs):
    for step in range(steps):

        offset = (step * batch_size) % (len(train_tweets) - batch_size)
        batch_tweets = train_tweets[offset : (offset + batch_size)]
        batch_labels = train_labels[offset : (offset + batch_size)]

        data = {tweets: batch_tweets, labels: batch_labels, keep_prob: 0.8}

        #run operations in graph
        _, loss_train, accuracy_train = session.run([optimizer, loss, accuracy], feed_dict=data)
    
        if (step % 50 == 0):
            test_loss = []
            test_accuracy = []
            print("Epoch:", epoch, "Step:", step)
            print("Summary step:",(summary_step))
            print("Train loss:", loss_train)
            print("Train accuracy:%.3f%%" % (accuracy_train*100))
            
            #tensorboard visualizations
            summary = session.run(merged_summary, feed_dict=data)
            train_writer.add_summary(summary, summary_step)
            
            for batch_num in range(int(len(test_tweets)/batch_size)):
                
                test_offset = (batch_num * batch_size) % (len(test_tweets) - batch_size)
                test_batch_tweets = test_tweets[test_offset : (test_offset + batch_size)]
                test_batch_labels = test_labels[test_offset : (test_offset + batch_size)]

                data_testing = {tweets: test_batch_tweets, labels: test_batch_labels, keep_prob: 1.0}

                loss_test, accuracy_test = session.run([loss, accuracy], feed_dict=data_testing)

                test_loss.append(loss_test)
                test_accuracy.append(accuracy_test)
            
            
            test_offset = (random.randint(0,len(test_tweets) - batch_size) * batch_size) % (len(test_tweets) - batch_size)
            test_batch_tweets = test_tweets[test_offset : (test_offset + batch_size)]
            test_batch_labels = test_labels[test_offset : (test_offset + batch_size)]
            
            data_testing = {tweets: test_batch_tweets, labels: test_batch_labels, keep_prob: 1.0}

            #tensorboard visualizations
            test_summary = session.run(merged_summary, feed_dict=data_testing)
            test_writer.add_summary(test_summary, summary_step)
            
            summary_step += 50
            global_step += 50
            
            print("Test loss:%.3f" % np.mean(test_loss))
            print("Test accuracy:%.3f%%" % (np.mean(test_accuracy)*100))


saver.save(session, 'checkpoints/pretrained_bin_lstm.ckpt', global_step=summary_step)

Epoch: 0 Step: 0
Summary step: 0
Train loss: 0.744185
Train accuracy:45.703%
Test loss:3.210
Test accuracy:50.080%
Epoch: 0 Step: 50
Summary step: 50
Train loss: 0.528763
Train accuracy:72.656%
Test loss:0.516
Test accuracy:74.102%
Epoch: 0 Step: 100
Summary step: 100
Train loss: 0.512814
Train accuracy:75.781%
Test loss:0.510
Test accuracy:74.705%
Epoch: 0 Step: 150
Summary step: 150
Train loss: 0.494052
Train accuracy:75.781%
Test loss:0.478
Test accuracy:76.576%
Epoch: 0 Step: 200
Summary step: 200
Train loss: 0.459193
Train accuracy:77.148%
Test loss:0.481
Test accuracy:76.618%
Epoch: 1 Step: 0
Summary step: 250
Train loss: 0.482597
Train accuracy:77.148%
Test loss:0.466
Test accuracy:77.476%
Epoch: 1 Step: 50
Summary step: 300
Train loss: 0.456263
Train accuracy:77.930%
Test loss:0.502
Test accuracy:75.798%
Epoch: 1 Step: 100
Summary step: 350
Train loss: 0.478091
Train accuracy:77.734%
Test loss:0.458
Test accuracy:78.010%
Epoch: 1 Step: 150
Summary step: 400
Train loss: 0.449022

Epoch: 13 Step: 200
Summary step: 3450
Train loss: 0.293495
Train accuracy:87.109%
Test loss:0.415
Test accuracy:81.632%
Epoch: 14 Step: 0
Summary step: 3500
Train loss: 0.323399
Train accuracy:85.352%
Test loss:0.423
Test accuracy:80.781%
Epoch: 14 Step: 50
Summary step: 3550
Train loss: 0.302794
Train accuracy:86.719%
Test loss:0.421
Test accuracy:81.529%
Epoch: 14 Step: 100
Summary step: 3600
Train loss: 0.360516
Train accuracy:83.594%
Test loss:0.417
Test accuracy:80.919%
Epoch: 14 Step: 150
Summary step: 3650
Train loss: 0.341331
Train accuracy:84.570%
Test loss:0.425
Test accuracy:81.160%
Epoch: 14 Step: 200
Summary step: 3700
Train loss: 0.293079
Train accuracy:86.523%
Test loss:0.437
Test accuracy:80.681%


'checkpoints/pretrained_bin_lstm.ckpt-3750'

In [6]:
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import gensim
import numpy as np
import random, re
MAX_NB_WORDS=20

def tweetsToVec(raw_tweets):
	exclude = '!"$%&\'()*+,-./:;<=>?¿[\\]^_`{|}~'
	regex = re.compile('[%s]' % re.escape(exclude))

	tweets = []

	tknzr = TweetTokenizer()
	stop_words = set(stopwords.words('spanish'))

	#Cleaning tweets
	for tweet in raw_tweets:
		#remove url
		no_url = re.sub(r"https?\S+", "", tweet)
		#no punctuation
		no_pun = regex.sub('', no_url)
		#tokenize
		tokenized = tknzr.tokenize(no_pun)
		#remove stop words
		important_words=[]
		for word in tokenized:
			if not word[0] == '#':
				important_words.append(word)

		# remove words if more than limit

		tweets.append(important_words)

	# load http://crscardellino.me/SBWCE/ trained model
	model = gensim.models.KeyedVectors.load_word2vec_format('data_process/SBW-vectors-300-min5.bin', binary=True)

	shape = (len(tweets), MAX_NB_WORDS, 300)
	tweets_tensor = np.zeros(shape, dtype=np.float32)

	for i in range(len(tweets)):
		#vectorizing each word in the tweet with a vector shape = (300,)
		length = len(tweets[i])
		for f in range(length):
			word = tweets[i][f]
			if f >= MAX_NB_WORDS:
				continue
			#if is not in the vocabulary
			if word in model.wv.vocab:
				tweets_tensor[i][f] = model.wv[word]
			else:
				#if it is a mention vectorize a name, for example @michael123 -> would be Carlos
				if word[0] == '@':
					tweets_tensor[i][f] = model.wv[name()]
				#if not append the unknown token
				else:
					tweets_tensor[i][f] = model.wv['unk']
		#End of sentence token
		if length - 1 < MAX_NB_WORDS:
			tweets_tensor[i][length - 1] = model.wv['eos']

	if len(tweets) == 1:
		return tweets_tensor
	else:
		return tweets_tensor


In [11]:
import tensorflow as tf
import numpy as np

tf.reset_default_graph()

sess = tf.Session()    
#First let's load meta graph and restore weights
saver = tf.train.import_meta_graph('checkpoints/pretrained_bin_lstm.ckpt-3750.meta')
saver.restore(sess, tf.train.latest_checkpoint('./checkpoints'))

# print(sess.run('saved:0'))
# This will print 2, which is the value of bias that we saved

graph = tf.get_default_graph()
    
# Access saved Variables directly
tweets = graph.get_tensor_by_name("tweets:0")
# labels = graph.get_tensor_by_name("labels:0")
# keep_prob = graph.get_tensor_by_name("keep_prob:0")

# [print(tensor.name) for tensor in tf.get_default_graph().as_graph_def().node]

# #Now, access the op that you want to run. 
pred = graph.get_tensor_by_name("pred_op:0")
# loss = graph.get_tensor_by_name("loss/loss_op:0")
# acc  = graph.get_tensor_by_name("accuracy/acc_op:0")


data = {tweets: tweetsToVec(["Gran día para pasear en bici por Ka ciudad!","Esto se complica...","Me encanta la tarde","Se me acabaron las vacaciones","la puta madrea tengo un examen","Muy bueno el cumple de Feli"])}

pred_ = sess.run([pred], feed_dict=data)

print("Prediction:", pred_)


INFO:tensorflow:Restoring parameters from ./checkpoints/pretrained_bin_lstm.ckpt-3750
Prediction: [array([ 0.99933904,  0.10356666,  0.59075803,  0.02792856,  0.05727113,
        0.93869275], dtype=float32)]


In [15]:
import tensorflow as tf
import numpy as np

tf.reset_default_graph()

export_dir = "models-binary/"

sess=tf.Session()    
#First let's load meta graph and restore weights
saver = tf.train.import_meta_graph('checkpoints/pretrained_bin_lstm.ckpt-3750.meta')
saver.restore(sess, tf.train.latest_checkpoint('./checkpoints'))


graph = tf.get_default_graph()

tweets = graph.get_tensor_by_name("tweets:0")
prediction = graph.get_tensor_by_name("pred_op:0")

builder = tf.saved_model.builder.SavedModelBuilder(export_dir)

inputs  = tf.saved_model.utils.build_tensor_info(tweets)
outputs = tf.saved_model.utils.build_tensor_info(prediction)

legacy_init_op = tf.group(tf.tables_initializer(), name='legacy_init_op')

prediction_signature = (
tf.saved_model.signature_def_utils.build_signature_def(
    inputs={'tweets': inputs},
    outputs={'scores': outputs},
    method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME))

builder.add_meta_graph_and_variables(sess,[tf.saved_model.tag_constants.SERVING],
    signature_def_map={
        tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: prediction_signature
    },legacy_init_op=legacy_init_op)


builder.save()

INFO:tensorflow:Restoring parameters from ./checkpoints/pretrained_bin_lstm.ckpt-3750
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'models-binary/saved_model.pb'


b'models-binary/saved_model.pb'

In [13]:
import tensorflow as tf
import gensim

tf.reset_default_graph()
model = gensim.models.KeyedVectors.load_word2vec_format('data_process/SBW-vectors-300-min5.bin', binary=True)

embedding_matrix = np.zeros((len(model.wv.vocab), 300))
for i in range(len(model.wv.vocab)):
    embedding_vector = model.wv[model.wv.index2word[i]]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        
# model = gensim.models.KeyedVectors.load_word2vec_format('data_process/SBW-vectors-300-min5.bin', binary=True)
# word = tf.Variable("hola")
# look = tf.nn.embedding_lookup(model.wv.index2word, 2)

# with tf.Session() as sess:
#     res = sess.run(look)
#     print(res)


In [ ]:
Prediction: [array([[  9.99706328e-01,   2.93696095e-04],
       [  1.04312964e-01,   8.95687044e-01],
       [  5.05031824e-01,   4.94968146e-01]], dtype=float32)]